<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/binaryoption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

In [ ]:
import QuantLib as ql
import unittest

# Helper function to convert Barrier::Type enum to string for reporting
def barrier_type_to_string(barrier_type: ql.Barrier.Type) -> str:
    if barrier_type == ql.Barrier.DownIn:
        return "Down-and-in"
    elif barrier_type == ql.Barrier.UpIn:
        return "Up-and-in"
    elif barrier_type == ql.Barrier.DownOut:
        return "Down-and-out"
    elif barrier_type == ql.Barrier.UpOut:
        return "Up-and-out"
    else:
        # Should not happen with QL enums if they are exhaustive
        raise ValueError(f"Unknown barrier type: {barrier_type}")

# Helper function to convert Payoff type to string for reporting
def payoff_type_to_string(payoff: ql.Payoff) -> str:
    if isinstance(payoff, ql.CashOrNothingPayoff):
        return "cash-or-nothing"
    elif isinstance(payoff, ql.AssetOrNothingPayoff):
        return "asset-or-nothing"
    elif isinstance(payoff, ql.PlainVanillaPayoff):
        return "vanilla"
    # Attempt to get name for other payoff types
    elif hasattr(payoff, 'name') and callable(getattr(payoff, 'name')):
        return payoff.name()
    else:
        return str(type(payoff).__name__)


class BinaryOptionTests(unittest.TestCase):

    def _report_failure(self, greek_name, payoff, exercise, barrier_type_enum,
                        barrier, s, q, r, today, v, expected, calculated, error, tolerance):
        # Format rates and vol as percentages for readability, similar to C++ io::rate/volatility
        q_str = f"{q * 100:.6f} %"
        r_str = f"{r * 100:.6f} %"
        v_str = f"{v * 100:.6f} %"

        # Get OptionType (Call/Put) from payoff
        option_type_str = str(payoff.optionType()) # e.g., "OptionType.Call"

        msg = (
            f"{option_type_str} option with "
            f"{barrier_type_to_string(barrier_type_enum)} barrier type:\n"
            f"    barrier:          {barrier}\n"
            f"{payoff_type_to_string(payoff)} payoff:\n"
            f"    spot value:       {s}\n"
            f"    strike:           {payoff.strike()}\n"
            f"    dividend yield:   {q_str}\n"
            f"    risk-free rate:   {r_str}\n"
            f"    reference date:   {today}\n"
            f"    maturity:         {exercise.lastDate()}\n"
            f"    volatility:       {v_str}\n\n"
            f"    expected   {greek_name}: {expected:.4f}\n"
            f"    calculated {greek_name}: {calculated:.4f}\n"
            f"    error:            {error:.4e}\n"
            f"    tolerance:        {tolerance:.1e}\n"
        )
        self.fail(msg)

    def testCashOrNothingHaugValues(self):
        print("Testing cash-or-nothing barrier options against Haug's values...")

        # Data from C++ test:
        # barrierType, barrier, cash, type, strike, s, q, r, t, v, result, tol
        values_data = [
            [ql.Barrier.DownIn,  100.00, 15.00, ql.Option.Call, 102.00, 105.00, 0.00, 0.10, 0.5, 0.20,  4.9289, 1e-4 ],
            [ql.Barrier.DownIn,  100.00, 15.00, ql.Option.Call,  98.00, 105.00, 0.00, 0.10, 0.5, 0.20,  6.2150, 1e-4 ],
            [ql.Barrier.UpIn,    100.00, 15.00, ql.Option.Call, 102.00,  95.00, 0.00, 0.10, 0.5, 0.20,  5.8926, 1e-4 ],
            [ql.Barrier.UpIn,    100.00, 15.00, ql.Option.Call,  98.00,  95.00, 0.00, 0.10, 0.5, 0.20,  7.4519, 1e-4 ],
            [ql.Barrier.DownIn,  100.00, 15.00, ql.Option.Put,  102.00, 105.00, 0.00, 0.10, 0.5, 0.20,  4.4314, 1e-4 ],
            [ql.Barrier.DownIn,  100.00, 15.00, ql.Option.Put,   98.00, 105.00, 0.00, 0.10, 0.5, 0.20,  3.1454, 1e-4 ],
            [ql.Barrier.UpIn,    100.00, 15.00, ql.Option.Put,  102.00,  95.00, 0.00, 0.10, 0.5, 0.20,  5.3297, 1e-4 ],
            [ql.Barrier.UpIn,    100.00, 15.00, ql.Option.Put,   98.00,  95.00, 0.00, 0.10, 0.5, 0.20,  3.7704, 1e-4 ],
            [ql.Barrier.DownOut, 100.00, 15.00, ql.Option.Call, 102.00, 105.00, 0.00, 0.10, 0.5, 0.20,  4.8758, 1e-4 ],
            [ql.Barrier.DownOut, 100.00, 15.00, ql.Option.Call,  98.00, 105.00, 0.00, 0.10, 0.5, 0.20,  4.9081, 1e-4 ],
            [ql.Barrier.UpOut,   100.00, 15.00, ql.Option.Call, 102.00,  95.00, 0.00, 0.10, 0.5, 0.20,  0.0000, 1e-4 ],
            [ql.Barrier.UpOut,   100.00, 15.00, ql.Option.Call,  98.00,  95.00, 0.00, 0.10, 0.5, 0.20,  0.0407, 1e-4 ],
            [ql.Barrier.DownOut, 100.00, 15.00, ql.Option.Put,  102.00, 105.00, 0.00, 0.10, 0.5, 0.20,  0.0323, 1e-4 ],
            [ql.Barrier.DownOut, 100.00, 15.00, ql.Option.Put,   98.00, 105.00, 0.00, 0.10, 0.5, 0.20,  0.0000, 1e-4 ],
            [ql.Barrier.UpOut,   100.00, 15.00, ql.Option.Put,  102.00,  95.00, 0.00, 0.10, 0.5, 0.20,  3.0461, 1e-4 ],
            [ql.Barrier.UpOut,   100.00, 15.00, ql.Option.Put,   98.00,  95.00, 0.00, 0.10, 0.5, 0.20,  3.0054, 1e-4 ],
            [ql.Barrier.UpIn,    100.00, 15.00, ql.Option.Call, 102.00,  95.00,-0.14, 0.10, 0.5, 0.20,  8.6806, 1e-4 ],
            [ql.Barrier.UpIn,    100.00, 15.00, ql.Option.Call, 102.00,  95.00, 0.03, 0.10, 0.5, 0.20,  5.3112, 1e-4 ],
            [ql.Barrier.DownIn,  100.00, 15.00, ql.Option.Call,  98.00,  95.00, 0.00, 0.10, 0.5, 0.20,  7.4926, 1e-4 ],
            [ql.Barrier.UpIn,    100.00, 15.00, ql.Option.Call,  98.00, 105.00, 0.00, 0.10, 0.5, 0.20, 11.1231, 1e-4 ],
            [ql.Barrier.DownIn,  100.00, 15.00, ql.Option.Put,  102.00,  98.00, 0.00, 0.10, 0.5, 0.20,  7.1344, 1e-4 ],
            [ql.Barrier.UpIn,    100.00, 15.00, ql.Option.Put,  102.00, 101.00, 0.00, 0.10, 0.5, 0.20,  5.9299, 1e-4 ],
            [ql.Barrier.DownOut, 100.00, 15.00, ql.Option.Call,  98.00,  99.00, 0.00, 0.10, 0.5, 0.20,  0.0000, 1e-4 ],
            [ql.Barrier.UpOut,   100.00, 15.00, ql.Option.Call,  98.00, 101.00, 0.00, 0.10, 0.5, 0.20,  0.0000, 1e-4 ],
            [ql.Barrier.DownOut, 100.00, 15.00, ql.Option.Put,   98.00,  99.00, 0.00, 0.10, 0.5, 0.20,  0.0000, 1e-4 ],
            [ql.Barrier.UpOut,   100.00, 15.00, ql.Option.Put,   98.00, 101.00, 0.00, 0.10, 0.5, 0.20,  0.0000, 1e-4 ],
        ]

        dc = ql.Actual360()
        # Use a fixed date for 'today' (evaluation date) to ensure reproducibility.
        fixed_today = ql.Date(22, ql.May, 2017)

        original_eval_date = ql.Settings.instance().evaluationDate
        ql.Settings.instance().evaluationDate = fixed_today

        # Setup reusable market data objects
        spot_quote = ql.SimpleQuote(0.0) # Initial value doesn't matter, will be set per case
        q_rate_quote = ql.SimpleQuote(0.0)
        r_rate_quote = ql.SimpleQuote(0.0)
        vol_quote = ql.SimpleQuote(0.0)

        q_ts_handle = ql.YieldTermStructureHandle(
            ql.FlatForward(fixed_today, ql.QuoteHandle(q_rate_quote), dc))
        r_ts_handle = ql.YieldTermStructureHandle(
            ql.FlatForward(fixed_today, ql.QuoteHandle(r_rate_quote), dc))
        vol_ts_handle = ql.BlackVolTermStructureHandle(
            ql.BlackConstantVol(fixed_today, ql.TARGET(), ql.QuoteHandle(vol_quote), dc))

        process = ql.BlackScholesMertonProcess(
            ql.QuoteHandle(spot_quote), q_ts_handle, r_ts_handle, vol_ts_handle)

        engine = ql.AnalyticBinaryBarrierEngine(process)

        for i, data_row in enumerate(values_data):
            (barrier_type, barrier_val, cash_val, option_type, strike_val,
             s_val, q_val, r_val, t_val, v_val, expected_npv, tol_val) = data_row

            spot_quote.setValue(s_val)
            q_rate_quote.setValue(q_val)
            r_rate_quote.setValue(r_val)
            vol_quote.setValue(v_val)

            payoff = ql.CashOrNothingPayoff(option_type, strike_val, cash_val)

            # C++ timeToDays(t) = Integer(t*365+0.5)
            days_to_maturity = int(t_val * 365.0 + 0.5)
            ex_date = fixed_today + ql.Period(days_to_maturity, ql.Days)
            # AnalyticBinaryBarrierEngine expects EuropeanExercise
            exercise = ql.EuropeanExercise(ex_date)

            # Rebate is 0 as per C++ code
            rebate = 0.0
            option = ql.BarrierOption(barrier_type, barrier_val, rebate, payoff, exercise)
            option.setPricingEngine(engine)

            calculated_npv = option.NPV()
            error = abs(calculated_npv - expected_npv)

            if error > tol_val:
                self._report_failure("value", payoff, exercise, barrier_type,
                                     barrier_val, s_val, q_val, r_val, fixed_today, v_val,
                                     expected_npv, calculated_npv, error, tol_val)

            # self.assertAlmostEqual can also be used, but _report_failure gives a more detailed message
            self.assertAlmostEqual(calculated_npv, expected_npv, delta=tol_val,
                                   msg=f"CashOrNothing - Case {i+1} failed.")

        ql.Settings.instance().evaluationDate = original_eval_date


    def testAssetOrNothingHaugValues(self):
        print("Testing asset-or-nothing barrier options against Haug's values...")

        values_data = [
            # barrierType, barrier, cash (placeholder), type, strike, s, q, r, t, v, result, tol
            [ql.Barrier.DownIn,  100.00,  0.00, ql.Option.Call, 102.00, 105.00, 0.00, 0.10, 0.5, 0.20, 37.2782, 1e-4 ],
            [ql.Barrier.DownIn,  100.00,  0.00, ql.Option.Call,  98.00, 105.00, 0.00, 0.10, 0.5, 0.20, 45.8530, 1e-4 ],
            [ql.Barrier.UpIn,    100.00,  0.00, ql.Option.Call, 102.00,  95.00, 0.00, 0.10, 0.5, 0.20, 44.5294, 1e-4 ],
            [ql.Barrier.UpIn,    100.00,  0.00, ql.Option.Call,  98.00,  95.00, 0.00, 0.10, 0.5, 0.20, 54.9262, 1e-4 ],
            [ql.Barrier.DownIn,  100.00,  0.00, ql.Option.Put,  102.00, 105.00, 0.00, 0.10, 0.5, 0.20, 27.5644, 1e-4 ],
            [ql.Barrier.DownIn,  100.00,  0.00, ql.Option.Put,   98.00, 105.00, 0.00, 0.10, 0.5, 0.20, 18.9896, 1e-4 ],
            [ql.Barrier.UpIn,    100.00,  0.00, ql.Option.Put,  102.00,  95.00, 0.00, 0.10, 0.5, 0.20, 33.1723, 1e-4 ],
            [ql.Barrier.UpIn,    100.00,  0.00, ql.Option.Put,   98.00,  95.00, 0.00, 0.10, 0.5, 0.20, 22.7755, 1e-4 ],
            [ql.Barrier.DownOut, 100.00,  0.00, ql.Option.Call, 102.00, 105.00, 0.00, 0.10, 0.5, 0.20, 39.9391, 1e-4 ],
            [ql.Barrier.DownOut, 100.00,  0.00, ql.Option.Call,  98.00, 105.00, 0.00, 0.10, 0.5, 0.20, 40.1574, 1e-4 ],
            [ql.Barrier.UpOut,   100.00,  0.00, ql.Option.Call, 102.00,  95.00, 0.00, 0.10, 0.5, 0.20,  0.0000, 1e-4 ],
            [ql.Barrier.UpOut,   100.00,  0.00, ql.Option.Call,  98.00,  95.00, 0.00, 0.10, 0.5, 0.20,  0.2676, 1e-4 ],
            [ql.Barrier.DownOut, 100.00,  0.00, ql.Option.Put,  102.00, 105.00, 0.00, 0.10, 0.5, 0.20,  0.2183, 1e-4 ],
            [ql.Barrier.DownOut, 100.00,  0.00, ql.Option.Put,   98.00, 105.00, 0.00, 0.10, 0.5, 0.20,  0.0000, 1e-4 ],
            [ql.Barrier.UpOut,   100.00,  0.00, ql.Option.Put,  102.00,  95.00, 0.00, 0.10, 0.5, 0.20, 17.2983, 1e-4 ],
            [ql.Barrier.UpOut,   100.00,  0.00, ql.Option.Put,   98.00,  95.00, 0.00, 0.10, 0.5, 0.20, 17.0306, 1e-4 ],
        ]

        dc = ql.Actual360()
        fixed_today = ql.Date(22, ql.May, 2017)

        original_eval_date = ql.Settings.instance().evaluationDate
        ql.Settings.instance().evaluationDate = fixed_today

        spot_quote = ql.SimpleQuote(0.0)
        q_rate_quote = ql.SimpleQuote(0.0)
        r_rate_quote = ql.SimpleQuote(0.0)
        vol_quote = ql.SimpleQuote(0.0)

        q_ts_handle = ql.YieldTermStructureHandle(
            ql.FlatForward(fixed_today, ql.QuoteHandle(q_rate_quote), dc))
        r_ts_handle = ql.YieldTermStructureHandle(
            ql.FlatForward(fixed_today, ql.QuoteHandle(r_rate_quote), dc))
        vol_ts_handle = ql.BlackVolTermStructureHandle(
            ql.BlackConstantVol(fixed_today, ql.TARGET(), ql.QuoteHandle(vol_quote), dc))

        process = ql.BlackScholesMertonProcess(
            ql.QuoteHandle(spot_quote), q_ts_handle, r_ts_handle, vol_ts_handle)

        engine = ql.AnalyticBinaryBarrierEngine(process)

        for i, data_row in enumerate(values_data):
            (barrier_type, barrier_val, _, option_type, strike_val, # cash_val (idx 2) is ignored
             s_val, q_val, r_val, t_val, v_val, expected_npv, tol_val) = data_row

            spot_quote.setValue(s_val)
            q_rate_quote.setValue(q_val)
            r_rate_quote.setValue(r_val)
            vol_quote.setValue(v_val)

            payoff = ql.AssetOrNothingPayoff(option_type, strike_val)

            days_to_maturity = int(t_val * 365.0 + 0.5)
            ex_date = fixed_today + ql.Period(days_to_maturity, ql.Days)
            exercise = ql.EuropeanExercise(ex_date)

            rebate = 0.0
            option = ql.BarrierOption(barrier_type, barrier_val, rebate, payoff, exercise)
            option.setPricingEngine(engine)

            calculated_npv = option.NPV()
            error = abs(calculated_npv - expected_npv)

            if error > tol_val:
                self._report_failure("value", payoff, exercise, barrier_type,
                                     barrier_val, s_val, q_val, r_val, fixed_today, v_val,
                                     expected_npv, calculated_npv, error, tol_val)

            self.assertAlmostEqual(calculated_npv, expected_npv, delta=tol_val,
                                   msg=f"AssetOrNothing - Case {i+1} failed.")

        ql.Settings.instance().evaluationDate = original_eval_date


if __name__ == '__main__':
    print("Presolve testQuantLib.py ...") # Mimic C++ test runner message style
    unittest.main(argv=['first-arg-is-ignored'], exit=False)